# Word2Vec (Negative Sampling)

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
np.__version__, torch.__version__

('1.26.4', '2.5.1+cu121')

In [4]:
import matplotlib
matplotlib.__version__

'3.10.0'

In [5]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#import os
import os

os.chdir('/content/drive/MyDrive/_NLP/NLP-A1-That-s-What-I-LIKE-st125553')

## 1. Load data

In [7]:
# Load nltk
import nltk

# download news category dataset from nltk
nltk.download('brown') # download brown corpus
nltk.download('punkt') # download punkt for tokenization
nltk.download('punkt_tab') # download punkt_tab for tokenization

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [8]:
#1. tokenization
# import the news category dataset
from nltk.corpus import brown
corpus = brown.sents(categories='news')
corpus

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [9]:
#2. numeralization
#find unique words
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(corpus))) #all the words we have in the system - <UNK>

print(f"before vocabs_len: {len(vocabs)}")
vocabs.append('<UNK>')
print(f"after vocabs_len: {len(vocabs)}")

before vocabs_len: 14394
after vocabs_len: 14395


In [10]:
#create handy mapping between integer and word
word2index = {v:idx for idx, v in enumerate(vocabs)}
word2index['<UNK>']

14394

In [11]:
# vocabs.append('<UNK>')
# word2index['<UNK>'] = 6

In [12]:
index2word = {v:k for k, v in word2index.items()}
index2word[14394]

'<UNK>'

## 2. Prepare train data

In [13]:
#create pairs of center word, and outside word

def random_batch(batch_size, corpus, window_size=2):

    skipgrams = []

    #loop each corpus
    for doc in corpus:
        #look from the 2nd word until second last word
        for i in range(1, len(doc)-window_size):
            #center word
            center = word2index[doc[i]]
            #outside words = 2 words
            #outside = (word2index[doc[i-1]], word2index[doc[i+1]])
            outside = [word2index[doc[j]] for j in range(i - window_size, i + window_size + 1) if j != i]

            #for each of these two outside words, we gonna append to a list
            for each_out in outside:
                skipgrams.append([center, each_out])
                #center, outside1;   center, outside2

    random_index = np.random.choice(range(len(skipgrams)), batch_size, replace=False)

    inputs, labels = [], []
    for index in random_index:
        inputs.append([skipgrams[index][0]])
        labels.append([skipgrams[index][1]])

    return np.array(inputs), np.array(labels)

window_size=2
x, y = random_batch(2, corpus, window_size)

In [14]:
x.shape  #batch_size, 1

(2, 1)

In [15]:
x

array([[ 9468],
       [12728]])

In [16]:
y.shape  #batch_size 1

(2, 1)

## 3. Negative Sampling

### Unigram distribution

$$P(w)=U(w)^{3/4}/Z$$

In [17]:
z = 0.001

In [18]:
#count
from collections import Counter

word_count = Counter(flatten(corpus))
word_count

#get the total number of words
num_total_words = sum([c for w, c in word_count.items()])
num_total_words

100554

In [19]:
#vocabs

$$P(w)=U(w)^{3/4}/Z$$

In [20]:
unigram_table = []

for v in vocabs:
    uw = word_count[v] / num_total_words
    uw_alpha = int((uw ** 0.75) / z)
    unigram_table.extend([v] * uw_alpha)

Counter(unigram_table)

Counter({'public': 3,
         'them': 5,
         'hope': 1,
         'life': 1,
         'Family': 1,
         'team': 2,
         'plus': 1,
         'her': 5,
         'private': 1,
         'security': 1,
         'very': 2,
         'Georgia': 1,
         'all': 8,
         'out': 8,
         'independence': 1,
         'collection': 1,
         'July': 1,
         'Richard': 2,
         'against': 4,
         'find': 1,
         "doesn't": 1,
         'seems': 1,
         'open': 2,
         'she': 2,
         'council': 1,
         'primary': 1,
         'committee': 2,
         'Kennedy': 4,
         'people': 3,
         'military': 2,
         'daily': 1,
         'talk': 1,
         'Army': 1,
         'road': 1,
         'amount': 1,
         'new': 7,
         'statement': 1,
         "Kennedy's": 1,
         'work': 4,
         'North': 2,
         'Pittsburgh': 1,
         'Catholic': 1,
         'Christian': 1,
         'L.': 1,
         'knew': 1,
         'fall': 1,


## 4. Model

$$\mathbf{J}_{\text{neg-sample}}(\mathbf{v}_c,o,\mathbf{U})=-\log(\sigma(\mathbf{u}_o^T\mathbf{v}_c))-\sum_{k=1}^K\log(\sigma(-\mathbf{u}_k^T\mathbf{v}_c))$$

In [21]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index['<UNK>'], seq))
    return torch.LongTensor(idxs)

In [22]:
import random

def negative_sampling(targets, unigram_table, k):
    batch_size = targets.shape[0]
    neg_samples = []
    for i in range(batch_size):  #(1, k)
        target_index = targets[i].item()
        nsample      = []
        while (len(nsample) < k):
            neg = random.choice(unigram_table)
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))

    return torch.cat(neg_samples) #batch_size, k

In [23]:
batch_size = 2
x, y = random_batch(batch_size, corpus)
x_tensor = torch.LongTensor(x)
y_tensor = torch.LongTensor(y)

In [24]:
k = 5
neg_samples = negative_sampling(y_tensor, unigram_table, k)

In [25]:
y_tensor[1]

tensor([2501])

In [26]:
neg_samples[1]

tensor([ 7736, 13002,  3296,  2687, 12631])

$$\mathbf{J}_{\text{neg-sample}}(\mathbf{v}_c,o,\mathbf{U})=-\log(\sigma(\mathbf{u}_o^T\mathbf{v}_c))-\sum_{k=1}^K\log(\sigma(-\mathbf{u}_k^T\mathbf{v}_c))$$

In [27]:
class SkipgramNeg(nn.Module):

    def __init__(self, voc_size, emb_size, word2index):
        super(SkipgramNeg, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
        self.logsigmoid        = nn.LogSigmoid()
        self.word2index        = word2index

    def forward(self, center, outside, negative):
        #center, outside:  (bs, 1)
        #negative       :  (bs, k)

        center_embed   = self.embedding_center(center) #(bs, 1, emb_size)
        outside_embed  = self.embedding_outside(outside) #(bs, 1, emb_size)
        negative_embed = self.embedding_outside(negative) #(bs, k, emb_size)

        uovc           = outside_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, 1)
        ukvc           = -negative_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, k)
        ukvc_sum       = torch.sum(ukvc, 1).reshape(-1, 1) #(bs, 1)

        loss           = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)

        return -torch.mean(loss)

    def get_embed(self, word):
        try:
            index = word2index[word]
        except:
            index = word2index['<UNK>']

        word = torch.LongTensor([index])

        embed_c = self.embedding_center(word)
        embed_o = self.embedding_outside(word)
        embed   = (embed_c + embed_o) / 2

        return embed[0][0].item(), embed[0][1].item()

In [28]:
#test your model
emb_size = 2
voc_size = len(vocabs)
model = SkipgramNeg(voc_size, emb_size, word2index)

In [29]:
loss = model(x_tensor, y_tensor, neg_samples)

In [30]:
loss

tensor(1.2955, grad_fn=<NegBackward0>)

## 5. Training

In [31]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [33]:
import time
start_time = time.time()

num_epochs = 1000

for epoch in range(num_epochs):
    start = time.time()
    #get batch
    input_batch, label_batch = random_batch(batch_size, corpus)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)

    #predict
    neg_samples = negative_sampling(label_tensor, unigram_table, k)
    loss = model(input_tensor, label_tensor, neg_samples)

    #backprogate
    optimizer.zero_grad()
    loss.backward()

    #update alpha
    optimizer.step()
    end = time.time()
    epoch_mins, epoch_secs = epoch_time(start, end)

    #print the loss
    if (epoch + 1) % 100 == 0:
        print(f"Epoch: {epoch + 1} | Lost: {loss:.6f} | time: {epoch_mins}m {epoch_secs}s")

print(f"Final Loss: {loss:2.6f}")
end_time = time.time()
minutes, seconds = epoch_time(start_time, end_time)
print(f"Total time: {minutes:2.0f} minutes {seconds:2.0f} seconds")

Epoch: 100 | Lost: 1.050550 | time: 0m 0s
Epoch: 200 | Lost: 3.600679 | time: 0m 0s
Epoch: 300 | Lost: 0.914883 | time: 0m 0s
Epoch: 400 | Lost: 4.349196 | time: 0m 0s
Epoch: 500 | Lost: 1.157018 | time: 0m 0s
Epoch: 600 | Lost: 4.499390 | time: 0m 0s
Epoch: 700 | Lost: 1.026583 | time: 0m 0s
Epoch: 800 | Lost: 1.601909 | time: 0m 0s
Epoch: 900 | Lost: 2.472253 | time: 0m 0s
Epoch: 1000 | Lost: 1.236114 | time: 0m 0s
Final Loss: 1.236114
Total time: 12 minutes 23 seconds


## 6. Testing

In [34]:
vect = []

for word in vocabs:
    vect.append(model.get_embed(word))
vect = np.array(vect)

In [35]:
#scipy version
from scipy import spatial

def cos_sim(a, b):
    cos_sim = 1 - spatial.distance.cosine(a, b)  #distance = 1 - similarlity, because scipy only gives distance
    return cos_sim

def cos_sim_scores(vect_space, target_vect):
    scores = []
    for each_vect in vect_space:
        each_vect = tuple(each_vect)
        target_vect=tuple(target_vect)
        scores.append(cos_sim(target_vect, each_vect))

    return np.array(scores)

In [36]:
def similarity(model, data):
    words = data.split(" ")

    embed0 = np.array(model.get_embed(words[0]))
    embed1 = np.array(model.get_embed(words[1]))
    embed2 = np.array(model.get_embed(words[2]))

    sim_vect = embed1 - embed0 + embed2

    sim_scores = cos_sim_scores(vect, sim_vect)
    max_score_idx = np.argmax(sim_scores)
    sim_word = index2word[max_score_idx]

    result = False
    if sim_word == words[3]:
        result = True

    return result

### Semantic Test

In [37]:
semantic_file = "data/word-test-semantic.txt"
# open file
with open(semantic_file, "r") as file:
    sem_file = file.readlines()
    #send semantic into vector

semantic = []
for sent in sem_file:
    semantic.append(sent.strip())

#semantic

In [38]:
sem_count = len(semantic)
#sem_total
sem_correct = 0
for sent in semantic:
    if similarity(model, sent):
        sem_correct += 1

In [39]:
sem_accuracy = sem_correct / sem_count
print(f"Semantic accuracy: {sem_accuracy:2.2f}")
print(f"Semantic correct: {sem_correct}")
print(f"Semantic count: {sem_count}")

Semantic accuracy: 0.00
Semantic correct: 0
Semantic count: 506


### Syntatic Test

In [40]:
syntatic_file = "data/word-test-syntatic.txt"
# open file
with open(syntatic_file, "r") as file:
    syn_file = file.readlines()

syntatic = []
for sent in syn_file:
    syntatic.append(sent.strip())
#syntatic

In [41]:
syn_count = len(syntatic)
syn_correct = 0
for sent in syntatic:
    if similarity(model, sent):
        syn_correct += 1

In [42]:
syn_accuracy = syn_correct / syn_count
print(f"Syntatic accuracy: {syn_accuracy:2.2f}")
print(f"Syntatic correct: {syn_correct}")
print(f"Syntatic count: {syn_count}")

Syntatic accuracy: 0.00
Syntatic correct: 0
Syntatic count: 1560


### Similarity Test


In [43]:
similarity_file = "data/wordsim353_sim_rel/wordsim_similarity_goldstandard.txt"
# open file
with open(similarity_file, "r") as file:
    sim_file = file.readlines()

similarity = []
for sent in sim_file:
    similarity.append(sent.strip())
#syntatic

In [44]:
def similarity_test(model, test_data):
    words = test_data.split("\t")

    embed0 = np.array(model.get_embed(words[0].strip()))
    embed1 = np.array(model.get_embed(words[1].strip()))

    model_result = embed1 @ embed0.T
    sim_result = float(words[2].strip())

    return sim_result, model_result

In [45]:
sim_scores = []
model_scores = []
for sent in similarity:
    sim_result, model_result = similarity_test(model, sent)

    sim_scores.append(sim_result)
    model_scores.append(model_result)

In [46]:
from scipy.stats import spearmanr

corr = spearmanr(sim_scores, model_scores)[0]

print(f"The correlation result is {corr:2.2f}.")

The correlation result is 0.14.


## 7. Saving the model

In [47]:
# Saving the model for testing
torch.save(model.state_dict(), 'app/models/w2v-skipgram-neg.model')

In [48]:
import pickle
neg_args = {
    'voc_size': voc_size,
    'emb_size': emb_size,
    'word2index': word2index,
}


In [49]:
import pickle
pickle.dump(neg_args, open('app/models/w2v-skipgram-neg.args', 'wb'))

In [50]:
neg_args = pickle.load(open('app/models/w2v-skipgram-neg.args', 'rb'))
model_neg = SkipgramNeg(**neg_args)
model_neg.load_state_dict(torch.load('app/models/w2v-skipgram-neg.model'))

<ipython-input-50-cd8207a5669d>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_neg.load_state_dict(torch.load('app/models/w2v-skipgram-neg.model'))


<All keys matched successfully>

In [51]:
# Test the model
model_neg.get_embed('The')

(-1.0371525287628174, 0.423246830701828)